In [ ]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
 
#download the treebank corpus from nltk
nltk.download('treebank')
 
#download the universal tagset from nltk
nltk.download('universal_tagset')
 
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
 
#print the first two sentences along with tags
print(nltk_data[:2])

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [ ]:
#print each word with its respective tag for first two sentences
for sent in nltk_data[:2]:
  for tuple in sent:
    print(tuple)

('Pierre', 'NOUN')
('Vinken', 'NOUN')
(',', '.')
('61', 'NUM')
('years', 'NOUN')
('old', 'ADJ')
(',', '.')
('will', 'VERB')
('join', 'VERB')
('the', 'DET')
('board', 'NOUN')
('as', 'ADP')
('a', 'DET')
('nonexecutive', 'ADJ')
('director', 'NOUN')
('Nov.', 'NOUN')
('29', 'NUM')
('.', '.')
('Mr.', 'NOUN')
('Vinken', 'NOUN')
('is', 'VERB')
('chairman', 'NOUN')
('of', 'ADP')
('Elsevier', 'NOUN')
('N.V.', 'NOUN')
(',', '.')
('the', 'DET')
('Dutch', 'NOUN')
('publishing', 'VERB')
('group', 'NOUN')
('.', '.')


In [ ]:
# split data into training and validation set in the ratio 80:20
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

In [ ]:
# create list of train and test tagged words
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

80310
20366


In [ ]:
# check some of the tagged words.
train_tagged_words[:5]

[('Drink', 'NOUN'),
 ('Carrier', 'NOUN'),
 ('Competes', 'VERB'),
 ('With', 'ADP'),
 ('Cartons', 'NOUN')]

In [ ]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

12
{'NOUN', 'VERB', 'PRT', 'ADP', 'ADV', 'PRON', 'DET', 'ADJ', 'X', '.', 'NUM', 'CONJ'}


In [ ]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return (count_w_given_tag, count_tag)

In [ ]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [ ]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[2.62344331e-01 1.49133503e-01 4.39345129e-02 1.76826611e-01
  1.68945398e-02 4.65906132e-03 1.31063312e-02 1.25838192e-02
  2.88252197e-02 2.40094051e-01 9.14395228e-03 4.24540639e-02]
 [1.10589318e-01 1.67955801e-01 3.06629837e-02 9.23572779e-02
  8.38858187e-02 3.55432779e-02 1.33609578e-01 6.63904250e-02
  2.15930015e-01 3.48066315e-02 2.28360966e-02 5.43278083e-03]
 [2.50489235e-01 4.01174158e-01 1.17416831e-03 1.95694715e-02
  9.39334650e-03 1.76125243e-02 1.01369865e-01 8.29745606e-02
  1.21330721e-02 4.50097844e-02 5.67514673e-02 2.34833662e-03]
 [3.23588967e-01 8.47886596e-03 1.26550242e-03 1.69577319e-02
  1.45532778e-02 6.96026310e-02 3.20931405e-01 1.07061505e-01
  3.45482156e-02 3.87243740e-02 6.32751212e-02 1.01240189e-03]
 [3.21955010e-02 3.39022487e-01 1.47401085e-02 1.19472459e-01
  8.14584941e-02 1.20248254e-02 7.13731572e-02 1.30721495e-01
  2.28859577e-02 1.39255241e-01 2.98681147e-02 6.98215654e-03]
 [2.12756261e-01 4.84738052e-01 1.41230067e-02 2.23234631e-02
  3

In [ ]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,NOUN,VERB,PRT,ADP,ADV,PRON,DET,ADJ,X,.,NUM,CONJ
NOUN,0.262344,0.149134,0.043935,0.176827,0.016895,0.004659,0.013106,0.012584,0.028825,0.240094,0.009144,0.042454
VERB,0.110589,0.167956,0.030663,0.092357,0.083886,0.035543,0.133610,0.066390,0.215930,0.034807,0.022836,0.005433
PRT,0.250489,0.401174,0.001174,0.019569,0.009393,0.017613,0.101370,0.082975,0.012133,0.045010,0.056751,0.002348
ADP,0.323589,0.008479,0.001266,0.016958,0.014553,0.069603,0.320931,0.107062,0.034548,0.038724,0.063275,0.001012
ADV,0.032196,0.339022,0.014740,0.119472,0.081458,0.012025,0.071373,0.130721,0.022886,0.139255,0.029868,0.006982
PRON,0.212756,0.484738,0.014123,0.022323,0.036902,0.006834,0.009567,0.070615,0.088383,0.041913,0.006834,0.005011
DET,0.635906,0.040247,0.000287,0.009918,0.012074,0.003306,0.006037,0.206411,0.045134,0.017393,0.022855,0.000431
ADJ,0.696893,0.011456,0.011456,0.080583,0.005243,0.000194,0.005243,0.063301,0.020971,0.066019,0.021748,0.016893
X,0.061695,0.206419,0.185086,0.142226,0.025754,0.054200,0.056890,0.017682,0.075726,0.160869,0.003075,0.010379
.,0.218539,0.089690,0.002789,0.092908,0.052569,0.068769,0.172192,0.046132,0.025641,0.092372,0.078210,0.060079
